In [1]:
import populator

In [2]:
data_dir = 'data'
# Second item in tuple is max batch size for requests.
exchanges = [('binance', 1000)]
tick_size = '1h'
start = '2000-01-01T00:00:00Z'

In [3]:
stables = ['TUSD/USDT', 'PAX/USDT', 'USDC/USDT', 'USDC/TUSD', 'USDS/TUSD', 'USDC/PAX', 'USDS/USDT']
populator.populate(data_dir, exchanges, stables, tick_size, start)
frames = [(pair, populator.load_data_as_frame(data_dir, 'binance', pair, tick_size, start)) for pair in stables]

Batch Size: 1000
Total Entries: 168336
Note: Entries will not exist if they predate exchange history.
Entries Processed: 162427
Entries Processed: 163427
Entries Processed: 164427
Entries Processed: 165437
Entries Processed: 166437
Entries Processed: 167437
Entries Processed: 168336
Scraping for data/binance/TUSD-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168336
Note: Entries will not exist if they predate exchange history.
Entries Processed: 165444
Entries Processed: 166444
Entries Processed: 167444
Entries Processed: 168336
Scraping for data/binance/PAX-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168336
Note: Entries will not exist if they predate exchange history.
Entries Processed: 167155
Entries Processed: 168155
Entries Processed: 168336
Scraping for data/binance/USDC-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168336
Note: Entries will not exist if they predate exchange h

In [4]:
import backtrader as bt

In [5]:
starting_cash = 10**5
class StableCoinStrategy(bt.Strategy):
    def next(self):
        if not self.position:
            self.buy(size=starting_cash / 2)
        elif self.data.close[0] > 1.0 and self.position.size > 1000:
            self.sell(size=1000)
        elif self.broker.getcash() > 1000 * self.data.close[0]:
            self.buy(size=1000)

In [6]:
def run_strategy(name, df):
    print('Running strategy for {}.'.format(name))
    cerebro = bt.Cerebro()
    cerebro.addstrategy(StableCoinStrategy)
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.00, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    cerebro.broker.setcash(starting_cash)
    cerebro.broker.setcommission(commission=0.001)
    sim = cerebro.run()
    
    print('Final Value: {}'.format(cerebro.broker.getvalue()))
    print('PNL %: {}'.format(100.0 * (cerebro.broker.getvalue() / starting_cash) - 100.0))
    print('Sharpe Ratio: {}'.format(sim[0].analyzers.sharpe.get_analysis()['sharperatio']))
    print('Max Drawdown: {}'.format(sim[0].analyzers.drawdown.get_analysis().max.drawdown))
    print()

In [7]:
for (name, df) in frames:
    run_strategy(name, df)

Running strategy for TUSD/USDT.
Final Value: 95648.38750000126
PNL %: -4.351612499998737
Sharpe Ratio: -2.0449445099871593
Max Drawdown: 4.855876699998815

Running strategy for PAX/USDT.
Final Value: 97294.53559999984
PNL %: -2.705464400000153
Sharpe Ratio: -3.046278876792461
Max Drawdown: 3.5031689000001496

Running strategy for USDC/USDT.
Final Value: 99287.12670000001
PNL %: -0.7128732999999983
Sharpe Ratio: -0.4947599793965925
Max Drawdown: 1.8941296047523668

Running strategy for USDC/TUSD.
Final Value: 99041.41519999999
PNL %: -0.9585848000000112
Sharpe Ratio: None
Max Drawdown: 1.078972200000062

Running strategy for USDS/TUSD.
Final Value: 99565.21840000009
PNL %: -0.4347815999999085
Sharpe Ratio: None
Max Drawdown: 0.5040815999999031

Running strategy for USDC/PAX.
Final Value: 98593.63210000006
PNL %: -1.4063678999999354
Sharpe Ratio: None
Max Drawdown: 1.4840876943133539

Running strategy for USDS/USDT.
Final Value: 99214.45770000013
PNL %: -0.7855422999998751
Sharpe Ratio: 